In [1]:
from birdy import WPSClient
from netCDF4 import Dataset
from pkg_resources import resource_filename
from wps_tools.testing import get_target_url
from wps_tools.utils import copy_http_content
from tempfile import NamedTemporaryFile
from datetime import date
import requests
import os

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "chickadee":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("chickadee")
print(f"Using chickadee on {url}")
url="http://localhost:5004/wps"

Using chickadee on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/chickadee/wps


In [3]:
chickadee = WPSClient(url)

In [4]:
# NBVAL_IGNORE_OUTPUT
chickadee.ca?

Signature:
chickadee.ca(
    gcm_file,
    obs_file=None,
    varname=None,
    indices_file=None,
    weight_file=None,
    num_cores='4',
    end_date=datetime.date(2005, 12, 31),
    loglevel='INFO',
)
Docstring:
Constructed Analogue (CA) downscaling algorithm

Parameters
----------
gcm_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of GCM simulations
obs_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of high-res gridded historical observations
varname : string
    Name of the NetCDF variable to downscale (e.g. 'tasmax')
num_cores : {'1', '2', '3', '4'}positiveInteger
    The number of cores to use for parallel execution
end_date : date
    Defines the end of the calibration period
indices_file : ComplexData:mimetype:`text/plain`
    File to store indices of analogue times steps
weight_file : ComplexData:mimetype:`text/plain`
    File to store weights of analogues
loglevel : {'C